In [ ]:
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import numpy as np
import math
import datetime
import copy

In [ ]:
from IPython.display import clear_output
for i in range(0, len(get_ipython().user_ns['In'])):
    clear_output(wait=True)

In [ ]:
# put your URL to folder with data here
path = 'C:\\Users\\Kossa\\Desktop\\respo_vision\\yolov7-pitch\\respo_dataset\\data_pitch_geom\\recruitment_task_pitch_geom'

In [ ]:
# and run 
labels_df = pd.read_csv(path+'\\df_keypoints.csv')

In [ ]:
# or 
labels_df = pd.read_csv('df_keypoints.csv')

In [ ]:
labels_df

In [ ]:
# check for nan 
labels_df.isna().any().any()

In [ ]:
# check if vis == 2 for x and y 
vis_check = labels_df[(labels_df['vis'] == 2) & ((labels_df['x'] == 0) | (labels_df['y'] == 0))]
vis_check

In [ ]:
to_check_list = [item.split('/')[-1] for item in vis_check['image_path']]

In [ ]:
to_check_list

## Plotting some random images

In [ ]:
random.seed(34)
random_img = random.sample(list(labels_df.loc[labels_df['dataset'] == 'train', 'image_path'].unique()), 100)

In [ ]:
random_img_names = [item.split('/')[-1] for item in random_img]

In [ ]:
test = labels_df[labels_df['image_path'] == 'images/train/c814ce8e31e077df861c2848fe14fb.jpg']
test

In [ ]:
labels_df.loc[labels_df['dataset'] == 'train', 'image_path']

In [ ]:
im = Image.open(path+'\\images\\train\\c814ce8e31e077df861c2848fe14fb.jpg')

In [ ]:
im.show()

In [ ]:
# optional, takes long time
# plot it here with annotation
%matplotlib inline

for img_name in random_img_names:

    print(img_name)

    img = mpimg.imread(path+f'\\images\\train\\{img_name}')
    imgplot = plt.imshow(img)

    temp_df = labels_df[labels_df['image_path'] == f'images/train/{img_name}']
    
    # Annotation
    for row in temp_df.itertuples():
        if row.vis:
            if row.x > 1000:
                sign_x = -1
            else:
                sign_x = 1
            if row.y < 500:
                sign_y = 1
            else:
                sign_y = -1
                
            plt.annotate(row.kid, xy =(row.x , row.y ),
                            xytext =(row.x + sign_x * 100, row.y + sign_y * 100), 
                            color = 'red', 
                            arrowprops = dict(facecolor ='red',
                                              headwidth = 5,
                                              headlength = 5,
                                              width = 1,
                                              shrink = 0.00001),)
    
    plt.show()

In [ ]:
#checking if those zeroes coordinates are valid

for img_name in to_check_list:

    print(img_name)

    img = mpimg.imread(path+f'\\images\\train\\{img_name}')
    imgplot = plt.imshow(img)

    temp_df = labels_df[labels_df['image_path'] == f'images/train/{img_name}']
    
    # Annotation
    for row in temp_df.itertuples():
        if row.vis:
            if row.x > 1000:
                sign_x = -1
            else:
                sign_x = 1
            if row.y < 500:
                sign_y = 1
            else:
                sign_y = -1
                
            plt.annotate(row.kid, xy =(row.x , row.y ),
                            xytext =(row.x + sign_x * 100, row.y + sign_y * 100), 
                            color = 'red', 
                            arrowprops = dict(facecolor ='red',
                                              headwidth = 5,
                                              headlength = 5,
                                              width = 1,
                                              shrink = 0.00001),)
    
    plt.show()

In [ ]:
vis_labels_df = labels_df[labels_df['vis'] == 2]
vis_labels_df

In [ ]:
vis_labels_df.groupby('kid').count()['x']

In [ ]:
#checking visible classes distribution
plt.figure(figsize = (16, 4))
plt.bar(vis_labels_df.groupby('kid').count()['x'].index, vis_labels_df.groupby('kid').count()['x'])
plt.xticks(rotation = 45)
plt.xticks(vis_labels_df.groupby('kid').count()['x'].index)
plt.xlabel('class no.')
plt.ylabel('n images')
plt.show()

## classes related to bottom corners (3 & 23) are very rare. Class related to intersection of bottom side of pitch and half line (10) is rare
## classes 24, 36, 37 are missing - this is correct according to the manual
## class 30 is missing but it should be present according to the manual!

## a270c12019e0005c93e28ca45b06dd.jpg - example of mislabeled file. Missing of label 30 is caused by mislabeling, not lack of images in the dataset!

In [ ]:
# To quickly and reliably input missing labels, I will assume that any of neighbor classes must be present in the image - 23, 29, 33 or 34
# Since classes pairs 3 - 23 and 9 - 29 have similar population, we can assume that population of class 30 will be similar to population of class 10 - that is about 150.
 
input_labels_paths = vis_labels_df.loc[(vis_labels_df['kid'] == 23) | (vis_labels_df['kid'] == 29) | (vis_labels_df['kid'] == 33) | (vis_labels_df['kid'] == 34), 'image_path'].unique()

In [ ]:
input_labels_paths= list(input_labels_paths)
input_labels_paths

In [ ]:
len(input_labels_paths)

In [ ]:
# takes long time but it is important, I plot images where class 30 maight be visible and note them

%matplotlib inline

for img_name in input_labels_paths:

    img_name = img_name.split('/')[-1]

    print(img_name)

    img = mpimg.imread(path+f'\\images\\train\\{img_name}')
    imgplot = plt.imshow(img)

    temp_df = labels_df[labels_df['image_path'] == f'images/train/{img_name}']
    
    # Annotation
    for row in temp_df.itertuples():
        if row.vis:
            if row.x > 1000:
                sign_x = -1
            else:
                sign_x = 1
            if row.y < 500:
                sign_y = 1
            else:
                sign_y = -1
                
            plt.annotate(row.kid, xy =(row.x , row.y ),
                            xytext =(row.x + sign_x * 100, row.y + sign_y * 100), 
                            color = 'red', 
                            arrowprops = dict(facecolor ='red',
                                              headwidth = 5,
                                              headlength = 5,
                                              width = 1,
                                              shrink = 0.00001),)
    
    plt.show()

In [ ]:
# files with class 30 to be fixed
fix_30_list = [
'9677b5248c7d2a391bb331263538c8.jpg',
'4fca4e0d72eef70eb308f4e92931b3.jpg',
'74f434bf605e8f6f5db0d409142bb7.jpg',
'fa9255414e8e18aa257670935fffba.jpg',
'60d86f695f30c394960023f391f50d.jpg',
'b73ab787697aec7a29675a88e771da.jpg',
'5fcf8d794a1c34024cc2c9efc285d9.jpg',
'b5d9188836d57ad32c0820713ab33e.jpg',
'41e9761846a0f10a8e6aecef400413.jpg',
'8dea69d0d05031d04d2b85af2a0705.jpg',
'2f1d6e937128e45c8257bf4b76db2e.jpg',
'cbf9e8794cd6aa406d2fc18946b89c.jpg',
'c05f429dbf4abeebeb4fa2e4937db8.jpg',
'2abe6d162668bb85bd8844103d58b3.jpg',
'ab65a0ffacb84533cf601a236f0920.jpg',
'38829ff54c7b1db3f49a31a1d5ece5.jpg',
'236e289ccb95432f1bbe07cf106c3b.jpg',
'2ef80b5e833ab9a2a3e269489dbf8d.jpg',
'a5941c096e19a477c13bb973e09df7.jpg',
'63cb5a548cd1032174d6605fabd290.jpg',
'4ef2046190bda2894769bf65e33ce7.jpg',
'3a4218907b274807c74be14032c901.jpg',
'78605986e67c9b89f1a807e858fe29.jpg',
'0676caa13e8abe3a568a2c61ef6e2d.jpg',
'f658ca5bdf4d8ee04fa3d648c9cc4f.jpg',
'7b304f4cd963b2fe573c3840315e79.jpg',
'66fa19216e23d7596b4d90da268e7c.jpg',# 33 confusion!!!!!!
'a1401f391b8f64656723df05c36b24.jpg',
'2791f191af7fca4b2e0b77cd050629.jpg',
'86a2a40a485feed061b6c5b5f961ee.jpg',
'c4f1f57482859c60ef0742691607f9.jpg',
'9ed1c1c9ee6cb5d5dbd61727d1c412.jpg',
'67d48e85a7792f3823b38c20a8ebf3.jpg',
'29c16eecccde846baba0b4f8bb5027.jpg',
'6546c56f291892f9b07a83eb2b1994.jpg',
'fdbab61c397896251067d8becc67e6.jpg',
'56c34b486748121bc22586949b0692.jpg',
'cdf10c1766c99298fed520400472a0.jpg',
'05a0d9cef3fb5fca8757e07b3bd867.jpg',
'a7267b1486cf8f8f8705e32452199e.jpg',
'ea158fb0fe07da57f02ff65684a647.jpg',
'335155589d1642be50e4de8953a6bf.jpg',
'2356ce9513bb1fe22364646639add7.jpg',
'9f8359d4bc8d2e63a0ce666f1936fd.jpg',
'9997dcea7cf66fe6e57fb36233208d.jpg',
'472fded23adc33c9a8bd5f7cbfd6b8.jpg',
'48d469d5006ce73f21e4be55a4f706.jpg',
'd4e0605f208a8365540f188d419ca6.jpg',
'e575b9605fb6f9fd6f729f393398b3.jpg',
'e25802bee03661511ddf0813904fea.jpg',
'a270c12019e0005c93e28ca45b06dd.jpg',
'a6eddc8c6d6d2afdbc871bf39ad49d.jpg',
'48f9c434173add06c222a44b3a95e4.jpg',
'62de8876e9adc7aa23af2ab13ec754.jpg',
'5f9a45153673ad24eb9f4ce6654ff1.jpg',
'0d0365deb03c4703e978f360d5ff72.jpg',
'710728f1a99225235e22abc62aeaab.jpg',
'02015eb88e33614b36170db06784d5.jpg',
'164bc44b724a585ae72e69ace0a10b.jpg',
'385acceeae6be647e931135c06bbea.jpg',
'2c4ff6598a91a53d7706ac185df094.jpg',
'92ffc7f0174f2e1e88ea739aef1f3f.jpg',
'a2cd5ab3d4985525865e565111eeeb.jpg',
'9bda0e6d253f33d80d4611f795a0c9.jpg',
'8f1bba299aa3484fc57a5a685e7be1.jpg',
'54fbd9767485472fa92a665b5ad9ff.jpg',
'2faca0c080867d91e5f7991b19f845.jpg',
'172dca5da5144cdb86558b9a6f1308.jpg',
'6e1f06d09ea276cc6c2e0d233facc3.jpg',
'd1f64e3430882e41de16b6b879fd9b.jpg',
'09f0bf76cd672ba841c926d8fdbfc8.jpg',
'03cc1aaab2c0985634cf4fa638b63a.jpg',
'cee6864c0ab20423896373ee8328d3.jpg',
'245f3357b3f3c7e97278610664ab8b.jpg',  ### specific orientation
'e681aa8ce2ce22486fd5d6d5870d2d.jpg',
'aef893839b0d08bc307f6c5b581c95.jpg',
'db6d9c9a386a306a65ed3939b65816.jpg',
'33b745960e851d2fd95e4abd6be141.jpg',
'0333a911c259ffb96e045cc998657f.jpg',
'75082ed4b6d5cf01ca376bbf67bcd8.jpg',
'00e6fafafd740664bef9aa4f387359.jpg',
'2c90e1733e23e5e56dbd63db45c63d.jpg',
'24e341f3e6399b9ad5864374a2a3c9.jpg',
'879b988c4b4d8f07709e2fd60510cf.jpg',
'05774d01da335be8ca3e263d622fdb.jpg',
'b5c506b0396c461aacf3c97e64afcf.jpg',
'a64c0ae39bb932c6a2eee247170c61.jpg',
'57de493eb70d824253227ddddbad86.jpg',
'17c0fd4742d6f872316ee1dd4959b8.jpg',
'21a05750e2d5fda4e77ebb179b86ea.jpg',
'1d2f4e3883651bb8a261cd1df3fcb6.jpg', ### specific orientation
'94b49ca68ed419da8ddbccd73dfd03.jpg', ### specific orientation
'fc8446a9d0ccde9c5d6cb10c6d3883.jpg',
'679fe17d8e3159e151938b5d8b3d83.jpg',
'b31a75c2688625e447879c620ac151.jpg',
'98b8d1e43aa64028280da5c354d459.jpg',
'65342549664333e6167919ff91fe47.jpg',
'06852b013d652b6850cba20fff83d8.jpg',
'bfc68987f3a82449d5533cb046deef.jpg',
'6ac91cbebe6df9a2669b1e0d008bbf.jpg',
'41375ecb2f0390f4328cb1c9a94179.jpg',
'baf7ca6c847404d9962ed81260c5c7.jpg',
'a1f6ede2ad8d82eeaf59b2dc022da8.jpg',
'28576c4dc0ff8c7082b63e6da72442.jpg',
'96c179f7e57d9e6204bde75aac9778.jpg',
'316777dee32535be262f63de39ca2d.jpg',
'b8f00314e9c80fc3d2a609adb41e63.jpg',
'f347fafdc58e6081b56ab63008dcfc.jpg',
'2869f77b076470b28b5f1b26b9a8df.jpg',
'665eed70d66d36754e1f4ff3688e6c.jpg',
'f1f9b766b7eff4d6de3792a89184fd.jpg',
'c91f4f418ead50690954cf487bf877.jpg',
'b2824f7190be843397ef4e2cd5cbdf.jpg',
'1e21d02ee9d25d2318964fd4bae782.jpg',
'c9819a80d876f2b4f9464a067eb65a.jpg',
'fc673285d41e0ef79c9bd8c887565c.jpg',
'32fcc238f33bb7165f3fa86c17620d.jpg',
'76b8e63b273ab668dfcef2805d92c5.jpg',
'10642681aebe3a3312d4fe3f6f4f1f.jpg',
]

In [ ]:
# plotting single image wit annotation
%matplotlib inline

for img_name in ['b31a75c2688625e447879c620ac151.jpg']:

    img_name = img_name.split('/')[-1]

    print(img_name)

    img = mpimg.imread(path+f'\\images\\train\\{img_name}')
    imgplot = plt.imshow(img)

    temp_df = labels_df[labels_df['image_path'] == f'images/train/{img_name}']
    
    # Annotation
    for row in temp_df.itertuples():
        if row.vis:
            if row.x > 1000:
                sign_x = -1
            else:
                sign_x = 1
            if row.y < 500:
                sign_y = 1
            else:
                sign_y = -1
                
            plt.annotate(row.kid, xy =(row.x , row.y ),
                            xytext =(row.x + sign_x * 100, row.y + sign_y * 100), 
                            color = 'red', 
                            arrowprops = dict(facecolor ='red',
                                              headwidth = 5,
                                              headlength = 5,
                                              width = 1,
                                              shrink = 0.00001),)
    
    plt.show()

In [ ]:
len(fix_30_list)
# similar population like class 10 - other side of the pitch

In [ ]:
labels_df

In [ ]:
filename = '9677b5248c7d2a391bb331263538c8.jpg'
labels_df[labels_df['image_path'] == f'images/train/{filename}']

In [ ]:
def line(p1, p2):
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

def intersection(L1, L2):
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return (x,y)
    else:
        return False

def get_point(filename, no_kid):

    _temp_df_loc = labels_df.loc[labels_df['image_path'] == f'images/train/{filename}', :]
    return (_temp_df_loc.loc[_temp_df_loc['kid'] == no_kid, 'x'].values[0], _temp_df_loc.loc[_temp_df_loc['kid'] == no_kid, 'y'].values[0])


def monte_carlo_find_30(filename, max_iter = 300000, m = 3000, s = 30, l_bound = 0, r_bound = 1920):
    '''
    A1 x + B1 y = C1
    A2 x + B2 y = C2
    Are parallel if and only if A1*B2 - B1*A2 == 0.
    I assume that line 33 - 34 is parallel to 29 - 30. In fact it is not, but let's hope its close enough
    PS It does not work well
    '''
    print('boundaries: ',l_bound, r_bound)
    r_X = []
    r_Y = []
    r_i = []
    p_33 = get_point(filename, 33)
    p_34 = get_point(filename, 34)
    p_29 = get_point(filename, 29)
    A1, B1, C1 = line(p_29, r_P)
    A2, B2, C2 = line(p_33, p_34)
    for j in range(s):

        for i in range(max_iter):
            A,B,C = line(get_point(img_name, 31), get_point(img_name, 34)) # get A,B,C for line that includes points 31 and 34
            rand_x = random.randint(l_bound, r_bound) # get random X within boundaries
            rand_y = int((C - (A*rand_x)) / B) # calc Y so it lays on the 31-34 line
            r_P = (rand_x, rand_y)
            #print(rand_x, rand_y)
            
            # print(A1*B2 - B1*A2)
            
            if abs(A1*B2 - B1*A2) < m:
                print(r_P)
                r_X.append(rand_x)
                r_Y.append(rand_y)
                r_i.append(i)
                break

    return r_X, r_Y, r_i

def monte_carlo_find_30_v2(filename, max_iter = 300000, m = 0.8, s = 30, l_bound = 0, r_bound = 1920):
    '''
    Experiments between aspects
    PS It does not work well
    '''
    print('boundaries: ',l_bound, r_bound)
    r_X = []
    r_Y = []
    r_i = []
    p_29 = get_point(filename, 29)
    p_31 = get_point(filename, 31)
    p_32 = get_point(filename, 32)
    p_33 = get_point(filename, 33)
    p_34 = get_point(filename, 34)
    A1, B1, C1 = line(p_31, p_32 )
    A2, B2, C2 = line(p_33, p_34 )
    aspect_const = A1*B2 - B1*A2
    
    for j in range(s):

        for i in range(max_iter):
            A,B,C = line(get_point(img_name, 31), get_point(img_name, 34)) # get A,B,C for line that includes points 31 and 34
            rand_x = random.randint(l_bound, r_bound) # get random X within boundaries
            rand_y = int((C - (A*rand_x)) / B) # calc Y so it lays on the 31-34 line
            r_P = (rand_x, rand_y)
            # print(f'[{rand_x}, {rand_y}],')
            
            A3, B3, C3 = line(p_29, r_P)
            
            # print(A1*B2 - B1*A2)
            aspect_1 = A2*B3 - B2*A3
            aspect_2 = A1*B3 - B1*A3
            
            if aspect_1 < 0 and aspect_2 > 0 and abs(aspect_1) < aspect_const:
                print(r_P)
                r_X.append(rand_x)
                r_Y.append(rand_y)
                r_i.append(i)
                break

    return r_X, r_Y, r_i


def monte_carlo_find_30_v3(filename, max_iter = 300000, m = 0.5, n = 0.01, s = 30, l_bound = 0, r_bound = 1920):
    '''
    Experiments between -A/B - that is a in directional form
    PS It is decent!
    '''
    print('boundaries: ',l_bound, r_bound)
    r_X = []
    r_Y = []
    r_i = []
    p_29 = get_point(filename, 29)
    p_31 = get_point(filename, 31)
    p_32 = get_point(filename, 32)
    p_33 = get_point(filename, 33)
    p_34 = get_point(filename, 34)
    A1, B1, C1 = line(p_31, p_32 )
    A2, B2, C2 = line(p_33, p_34 )
    a1 = -A1/B1
    a2 = -A2/B2
    
    for j in range(s):

        for i in range(max_iter):
            A,B,C = line(get_point(filename, 31), get_point(filename, 34)) # get A,B,C for line that includes points 31 and 34
            rand_x = random.randint(l_bound, r_bound) # get random X within boundaries
            rand_y = int((C - (A*rand_x)) / B) # calc Y so it lays on the 31-34 line
            r_P = (rand_x, rand_y)

            A3, B3, C3 = line(p_29, r_P)
            a3  = -A3/B3
            a3_goto = a2 + (a1*m)
            
            #if abs(a3) > abs(a1) and abs(a3) > abs(a2) and abs(a3) < (abs(a1) + abs(a2))*m and  abs(a3) > (abs(a1) + abs(a2))*n:
            if a3 < 0 and abs(a3 - a3_goto) < n:
                print(r_P)
                r_X.append(rand_x)
                r_Y.append(rand_y)
                r_i.append(i)
                break

    return r_X, r_Y, r_i

def monte_carlo_find_30_v4(filename, max_iter = 300000, m = 0.01, s = 30, l_bound = 0, r_bound = 1920):
    '''
    Experiments between -A/B - that is a in directional form, this time checking for parallel
    PS It does not work so good

    '''
    print('boundaries: ',l_bound, r_bound)
    r_X = []
    r_Y = []
    r_i = []
    p_29 = get_point(filename, 29)
    p_31 = get_point(filename, 31)
    p_32 = get_point(filename, 32)
    p_33 = get_point(filename, 33)
    p_34 = get_point(filename, 34)
    # A1, B1, C1 = line(p_31, p_32 )
    A2, B2, C2 = line(p_33, p_34 )
    # a1 = -A1/B1
    a2 = -A2/B2
    
    for j in range(s):

        for i in range(max_iter):
            A,B,C = line(get_point(filename, 31), get_point(filename, 34)) # get A,B,C for line that includes points 31 and 34
            rand_x = random.randint(l_bound, r_bound) # get random X within boundaries
            rand_y = int((C - (A*rand_x)) / B) # calc Y so it lays on the 31-34 line
            r_P = (rand_x, rand_y)

            A3, B3, C3 = line(p_29, r_P)
            a3  = -A3/B3
            
            #if abs(a3) > abs(a1) and abs(a3) > abs(a2) and abs(a3) < (abs(a1) + abs(a2))*m and  abs(a3) > (abs(a1) + abs(a2))*n:
            if abs(a3) > abs(a2) and abs(a3) - abs(a2) < m:
                print(r_P)
                r_X.append(rand_x)
                r_Y.append(rand_y)
                r_i.append(i)
                break

    return r_X, r_Y, r_i


In [ ]:
#code for testing purpose only

%matplotlib inline
test_rP = [
(1608, 436),
(1626, 439),
(1611, 436),
(1608, 436),
(1624, 439),
(1617, 438),
(1617, 438),
(1614, 437),
(1625, 439),
(1626, 439),
]
for rx, ry in test_rP:
    for img_name in ['9677b5248c7d2a391bb331263538c8.jpg']:
    
        img_name = img_name.split('/')[-1]
    
        print(img_name)
    
        img = mpimg.imread(path+f'\\images\\train\\{img_name}')
        imgplot = plt.imshow(img)
    
        temp_df = labels_df[labels_df['image_path'] == f'images/train/{img_name}']
        
        # Annotation
        for row in temp_df.itertuples():
            if row.vis:
                if row.x > 1000:
                    sign_x = -1
                else:
                    sign_x = 1
                if row.y < 500:
                    sign_y = 1
                else:
                    sign_y = -1
                    
                plt.annotate(row.kid, xy =(row.x , row.y ),
                                xytext =(row.x + sign_x * 100, row.y + sign_y * 100), 
                                color = 'red', 
                                arrowprops = dict(facecolor ='red',
                                                  headwidth = 5,
                                                  headlength = 5,
                                                  width = 1,
                                                  shrink = 0.00001),)
    
        print(get_point(img_name, 31))
        print(get_point(img_name, 34))

        p_31 = get_point(img_name, 31)
        p_32 = get_point(img_name, 32)
        p_34 = get_point(img_name, 34)


        
        plt.plot([rx, p_31[0]], [ry, p_31[1]], color = 'red')
    
        p_33 = get_point(img_name, 33)
    
        plt.plot([p_33[0], p_34[0]], [p_33[1], p_34[1]], color = 'blue')
    
        p_29 = get_point(img_name, 29)
    
        plt.plot([rx, p_29[0]], [ry, p_29[1]], color = 'yellow')

        plt.plot([p_31[0], p_32[0]], [p_31[1], p_32[1]], color = 'blue')
    
        A1,B1,C1 = line(get_point(img_name, 31), get_point(img_name, 32))
        A2,B2,C2 = line(get_point(img_name, 33), get_point(img_name, 34))
        A3,B3,C3 = line(get_point(img_name, 29), [rx, ry])
        a1 = -A1/B1
        a2 = -A2/B2
        a3 = -A3/B3
        print(a1)
        print(a2)
        print(a3)
        print(abs(a1) + abs(a2))
        print(abs(a1) < (abs(a1) + abs(a2)))
        plt.show()

In [ ]:
now = datetime.datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

In [ ]:
# calculating approximated position of class 30 here, takes 10 - 15 min
outcome_dict = {}

for filename in fix_30_list:

    now = datetime.datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)
    print(filename)
    
    
    temp_df = labels_df.loc[labels_df['image_path'] == f'images/train/{filename}', :]
    vis_temp_df = temp_df[temp_df['vis']==2]

    _flag = False
    for kid in [29,31,32,33,34]:
        if kid not in vis_temp_df['kid'].values:
            outcome_dict[filename]= 'failed'
            _flag = True
            continue
    if _flag:
        continue
        
    r_X, r_Y, r_i = monte_carlo_find_30_v3(filename, max_iter = 300, m = 0.35, n = 0.03, s = 30, 
                    l_bound = get_point(filename, 34)[0], 
                    r_bound = 1920, 
                    )

    outcome_dict[filename]= [r_X, r_Y, r_i]

    X_30_mean = np.array(r_X).mean()
    Y_30_mean = np.array(r_Y).mean()
    
    print('XY_30: ',  X_30_mean, '+/-', np.array(r_X).std(), ', ', Y_30_mean , '+/-', np.array( r_Y).std())
    print('iter: ',  np.array(r_i).mean(), '+/-', np.array(r_i).std())
    
    # img_name = img_name.split('/')[-1]

    # print(img_name)

    img = mpimg.imread(path+f'\\images\\train\\{filename}')
    imgplot = plt.imshow(img)

    temp_df = labels_df[labels_df['image_path'] == f'images/train/{filename}']
    
    # Annotation
    for row in temp_df.itertuples():
        if row.vis:
            if row.x > 1000:
                sign_x = -1
            else:
                sign_x = 1
            if row.y < 500:
                sign_y = 1
            else:
                sign_y = -1
                
            plt.annotate(row.kid, xy =(row.x , row.y ),
                            xytext =(row.x + sign_x * 100, row.y + sign_y * 100), 
                            color = 'red', 
                            arrowprops = dict(facecolor ='red',
                                              headwidth = 5,
                                              headlength = 5,
                                              width = 1,
                                              shrink = 0.00001),)

    if X_30_mean > 1000:
        sign_x = -1
    else:
        sign_x = 1
    if Y_30_mean < 500:
        sign_y = 1
    else:
        sign_y = -1
                
    plt.annotate(30, xy =(X_30_mean , Y_30_mean ),
                    xytext =(X_30_mean + sign_x * 100, Y_30_mean + sign_y * 100), 
                    color = 'red', 
                    arrowprops = dict(facecolor ='red',
                                      headwidth = 5,
                                      headlength = 5,
                                      width = 1,
                                      shrink = 0.00001),)
    
    plt.show()


In [ ]:
failed_list = []
for key in outcome_dict:

    if outcome_dict[key] == 'failed':
        print('failed: ', key)
        failed_list.append(key)

In [ ]:
%matplotlib inline

for img_name in failed_list:

    img_name = img_name.split('/')[-1]

    print(img_name)

    img = mpimg.imread(path+f'\\images\\train\\{img_name}')
    imgplot = plt.imshow(img)

    temp_df = labels_df[labels_df['image_path'] == f'images/train/{img_name}']
    
    # Annotation
    for row in temp_df.itertuples():
        if row.vis:
            if row.x > 1000:
                sign_x = -1
            else:
                sign_x = 1
            if row.y < 500:
                sign_y = 1
            else:
                sign_y = -1
                
            plt.annotate(row.kid, xy =(row.x , row.y ),
                            xytext =(row.x + sign_x * 100, row.y + sign_y * 100), 
                            color = 'red', 
                            arrowprops = dict(facecolor ='red',
                                              headwidth = 5,
                                              headlength = 5,
                                              width = 1,
                                              shrink = 0.00001),)
    
    plt.show()

## these files are missing point 31 or 32 or both - these are needed to calculate approximate position of point 30.
## since there is few of them, they will be labelled manually

In [ ]:
# labelled manually (in Paint of course). In fact all missing cls 30 labels should be labelled manually with dedicated tool - such as labelImg
failed_dict = {}
failed_dict['4fca4e0d72eef70eb308f4e92931b3.jpg'] = [1369, 443]
failed_dict['fa9255414e8e18aa257670935fffba.jpg'] = [1005, 589]
failed_dict['a1401f391b8f64656723df05c36b24.jpg'] = [1055, 453]
failed_dict['2791f191af7fca4b2e0b77cd050629.jpg'] = [1105, 419]
failed_dict['67d48e85a7792f3823b38c20a8ebf3.jpg'] = [1110, 572]
failed_dict['a7267b1486cf8f8f8705e32452199e.jpg'] = [1611, 665]
failed_dict['ea158fb0fe07da57f02ff65684a647.jpg'] = [984, 641]
failed_dict['d4e0605f208a8365540f188d419ca6.jpg'] = [1186, 296]
failed_dict['e575b9605fb6f9fd6f729f393398b3.jpg'] = [1204,323 ]
failed_dict['92ffc7f0174f2e1e88ea739aef1f3f.jpg'] = [1155,699 ]
failed_dict['172dca5da5144cdb86558b9a6f1308.jpg'] = [1470,648 ]
failed_dict['24e341f3e6399b9ad5864374a2a3c9.jpg'] = [1128,618 ]
failed_dict['b5c506b0396c461aacf3c97e64afcf.jpg'] = [1188,532 ]
# failed_dict['17c0fd4742d6f872316ee1dd4959b8.jpg'] = [, ] no 30 here
failed_dict['65342549664333e6167919ff91fe47.jpg'] = [1661, 679]
failed_dict['06852b013d652b6850cba20fff83d8.jpg'] = [1125,434 ]
failed_dict['bfc68987f3a82449d5533cb046deef.jpg'] = [1332,412 ]
failed_dict['66fa19216e23d7596b4d90da268e7c.jpg'] = [1267,679]

In [ ]:
#check if labelled correctly

for filename in failed_dict.keys():

    
    temp_df = labels_df.loc[labels_df['image_path'] == f'images/train/{filename}', :]

    X_30_mean = failed_dict[filename][0]
    Y_30_mean = failed_dict[filename][1]


    img = mpimg.imread(path+f'\\images\\train\\{filename}')
    imgplot = plt.imshow(img)

    
    # Annotation
    for row in temp_df.itertuples():
        if row.vis:
            if row.x > 1000:
                sign_x = -1
            else:
                sign_x = 1
            if row.y < 500:
                sign_y = 1
            else:
                sign_y = -1
                
            plt.annotate(row.kid, xy =(row.x , row.y ),
                            xytext =(row.x + sign_x * 100, row.y + sign_y * 100), 
                            color = 'red', 
                            arrowprops = dict(facecolor ='red',
                                              headwidth = 5,
                                              headlength = 5,
                                              width = 1,
                                              shrink = 0.00001),)

    if X_30_mean > 1000:
        sign_x = -1
    else:
        sign_x = 1
    if Y_30_mean < 500:
        sign_y = 1
    else:
        sign_y = -1
                
    plt.annotate(30, xy =(X_30_mean , Y_30_mean ),
                    xytext =(X_30_mean + sign_x * 100, Y_30_mean + sign_y * 100), 
                    color = 'red', 
                    arrowprops = dict(facecolor ='red',
                                      headwidth = 5,
                                      headlength = 5,
                                      width = 1,
                                      shrink = 0.00001),)
    
    plt.show()

## ANOTHER NOTE: Classes 29 and 39 are sometimes confused! - a91e197d6178c45c647f84b81fa79e.jpg, cb58f23e323571467454d82b5e4b7e.jpg

In [ ]:
# optional
# randomly plotting 30 images for EACH class to determine possible further problems

k = 30
for i in range(1, 40):
    
    _tmp = list(vis_labels_df.loc[(vis_labels_df['kid'] == i), 'image_path'].unique())
                
    if len(_tmp) >= k:
        random_labels_paths = random.sample(_tmp, k)
    else:
        random_labels_paths = random.sample(_tmp, len(_tmp))

    print(f'*****************STARTING {i} ************************') 

    for img_name in random_labels_paths:
    
        img_name = img_name.split('/')[-1]
    
        print(img_name, ' ', i)
    
        img = mpimg.imread(path+f'\\images\\train\\{img_name}')
        imgplot = plt.imshow(img)
    
        temp_df = labels_df[labels_df['image_path'] == f'images/train/{img_name}']
        
        # Annotation
        for row in temp_df.itertuples():
            if row.vis:
                if row.x > 1000:
                    sign_x = -1
                else:
                    sign_x = 1
                if row.y < 500:
                    sign_y = 1
                else:
                    sign_y = -1
                    
                plt.annotate(row.kid, xy =(row.x , row.y ),
                                xytext =(row.x + sign_x * 100, row.y + sign_y * 100), 
                                color = 'red', 
                                arrowprops = dict(facecolor ='red',
                                                  headwidth = 5,
                                                  headlength = 5,
                                                  width = 1,
                                                  shrink = 0.00001),)
        
        plt.show()

#outcome_removed - too many images

In [ ]:
# rest of points are labelled correctly
# 38 needs to be changed to 39 in a4aae193d810ebde815016b779f0f0.jpg


In [ ]:
# when 29 is confused with 39, vis of 39 must be 0
# input_labels_paths = labels_df.loc[(vis_labels_df['kid'] == 23) | (vis_labels_df['kid'] == 29) | (vis_labels_df['kid'] == 33) | (vis_labels_df['kid'] == 34), 'image_path'].unique()

In [ ]:
filenames_with_29_n_no_39 = []
for filename in labels_df['image_path'].unique():
    temp_df = labels_df[labels_df['image_path'] == filename]
    if (temp_df.loc[temp_df['kid'] == 29, 'vis'].values[0] == 2) and (temp_df.loc[temp_df['kid'] == 39, 'vis'].values[0] == 0):
        filenames_with_29_n_no_39.append(filename)

# can be done faster with for filename in labels_df.groupby('image_path')


In [ ]:
filenames_with_29_n_no_39

In [ ]:
for img_name in filenames_with_29_n_no_39:

    img_name = img_name.split('/')[-1]

    print(img_name)

    img = mpimg.imread(path+f'\\images\\train\\{img_name}')
    imgplot = plt.imshow(img)

    temp_df = labels_df[labels_df['image_path'] == f'images/train/{img_name}']
    
    # Annotation
    for row in temp_df.itertuples():
        if row.vis:
            if row.x > 1000:
                sign_x = -1
            else:
                sign_x = 1
            if row.y < 500:
                sign_y = 1
            else:
                sign_y = -1
                
            plt.annotate(row.kid, xy =(row.x , row.y ),
                            xytext =(row.x + sign_x * 100, row.y + sign_y * 100), 
                            color = 'red', 
                            arrowprops = dict(facecolor ='red',
                                              headwidth = 5,
                                              headlength = 5,
                                              width = 1,
                                              shrink = 0.00001),)
    
    plt.show()

In [ ]:
#only 2 are wrong
# f9a1444f703b8dbbecc357c0cfb949.jpg
#a91e197d6178c45c647f84b81fa79e.jpg

In [ ]:
corrected_labels_df = copy.deepcopy(labels_df)

In [ ]:
corrected_labels_df

In [ ]:
# inputting corrected cls 30 values

for filename in outcome_dict:
    if filename == '66fa19216e23d7596b4d90da268e7c.jpg':
        continue
    if outcome_dict[filename] == 'failed':
        continue
    else:
        print(filename)
        x_30 = int(np.array(outcome_dict[filename][0]).mean())
        y_30 = int(np.array(outcome_dict[filename][1]).mean())
        print(x_30, y_30)
        corrected_labels_df.loc[
                        (corrected_labels_df['image_path'] == f'images/train/{filename}') & (corrected_labels_df['kid'] == 30), ['x','y']] = x_30, y_30
        

In [ ]:
corrected_labels_df[corrected_labels_df['image_path'] == f'images/train/6ac91cbebe6df9a2669b1e0d008bbf.jpg']

In [ ]:
for filename in failed_dict:

    print(filename)
    x_30 = failed_dict[filename][0]
    y_30 = failed_dict[filename][1]
    print(x_30, y_30)
    corrected_labels_df.loc[
                    (corrected_labels_df['image_path'] == f'images/train/{filename}') & (corrected_labels_df['kid'] == 30), ['x','y']] = x_30, y_30
        

In [ ]:
corrected_labels_df[corrected_labels_df['image_path'] == f'images/train/e575b9605fb6f9fd6f729f393398b3.jpg']

In [ ]:
for row in corrected_labels_df.itertuples():
    if not row.vis:
        if row.x and row.y:
           print(row) # all are 30

In [ ]:
#fixing vis
for row in corrected_labels_df.itertuples():
    if not row.vis:
        if row.x and row.y:
#            print(row) # all are 30
            corrected_labels_df.loc[row.Index, 'vis'] = 2


In [ ]:
corrected_labels_df[corrected_labels_df['image_path'] == f'images/train/e575b9605fb6f9fd6f729f393398b3.jpg']

In [ ]:
#files to fix

#only 2 are wrong
# f9a1444f703b8dbbecc357c0cfb949.jpg
#a91e197d6178c45c647f84b81fa79e.jpg


#'66fa19216e23d7596b4d90da268e7c.jpg',# 33 confusion!!!!!!

In [ ]:
filename = '66fa19216e23d7596b4d90da268e7c.jpg'


print(filename)

img = mpimg.imread(path+f'\\images\\train\\{filename}')
imgplot = plt.imshow(img)

temp_df = labels_df[labels_df['image_path'] == f'images/train/{filename}']

# Annotation
for row in temp_df.itertuples():
    if row.vis:
        if row.x > 1000:
            sign_x = -1
        else:
            sign_x = 1
        if row.y < 500:
            sign_y = 1
        else:
            sign_y = -1
            
        plt.annotate(row.kid, xy =(row.x , row.y ),
                        xytext =(row.x + sign_x * 100, row.y + sign_y * 100), 
                        color = 'red', 
                        arrowprops = dict(facecolor ='red',
                                          headwidth = 5,
                                          headlength = 5,
                                          width = 1,
                                          shrink = 0.00001),)

In [ ]:
corrected_labels_df[corrected_labels_df['image_path'] == f'images/train/f9a1444f703b8dbbecc357c0cfb949.jpg']

In [ ]:
corrected_labels_df.loc[
                (corrected_labels_df['image_path'] == f'images/train/{filename}') & (corrected_labels_df['kid'] == 39), ['x','y','vis']] = 1916, 452, 2
corrected_labels_df.loc[
                (corrected_labels_df['image_path'] == f'images/train/{filename}') & (corrected_labels_df['kid'] == 29), ['x','y','vis']] = 0,0,0

In [ ]:
corrected_labels_df[corrected_labels_df['image_path'] == f'images/train/{filename}']

In [ ]:
corrected_labels_df.loc[
                (corrected_labels_df['image_path'] == f'images/train/{filename}') & (corrected_labels_df['kid'] == 39), ['x','y','vis']] = 1850,	509, 2
corrected_labels_df.loc[
                (corrected_labels_df['image_path'] == f'images/train/{filename}') & (corrected_labels_df['kid'] == 29), ['x','y','vis']] = 0,0,0

In [ ]:
corrected_labels_df[corrected_labels_df['image_path'] == f'images/train/{filename}']

In [ ]:
corrected_labels_df[corrected_labels_df['image_path'] == f'images/train/{filename}']

In [ ]:

corrected_labels_df.loc[
                (corrected_labels_df['image_path'] == f'images/train/{filename}') & (corrected_labels_df['kid'] == 33), ['x','y','vis']] = 809, 629, 2

In [ ]:
corrected_labels_df[corrected_labels_df['image_path'] == f'images/train/{filename}']

In [ ]:
 # 'images/train/74f434bf605e8f6f5db0d409142bb7.jpg',

In [ ]:
corrected_labels_df.to_csv('df_keypoints_corrected.csv')

## files below are labelled in df_keypoints as train, but they are not labelled with x, y and vis. ['dataset'] should be 'test' for them

In [ ]:
no_vis_classes_val = ['a37d3123f9202186d20a7765ec2c0a.jpg',
 '934a5454a825420abe74ace1cbe2b0.jpg',
 'fffe2a9e755bf705c79753bf09b7f1.jpg',
 '73aec0a4b73935433808780a577f3c.jpg',
 '32e27114c613d5a58cc95b3f6262ab.jpg',
 '2e031ce6ec3a9b7a45d70bcdf7618e.jpg',
 '03a17019ec63792e8316633d50d109.jpg',
 '3e6cbfd6d83cc24393092a74522062.jpg',
 '07b7b54074da586b6c0effd7f5ba9e.jpg',
 '5ad1adfad421cb8bd6ced67369cd38.jpg',
 '370bcfad2192293a0a25daf431d7d4.jpg',
 'ae1ba09379c7277fa88c5cff12c398.jpg',
 '7db8876376346a856b26cf799e97dd.jpg']

In [ ]:
no_vis_classes_train = ['95bf87b5b9a624d3c29404222e5bc5.jpg',
 '36bdfe4751cddb053043ec07249d91.jpg',
 '03c0b0afbe2f26db857d197682535f.jpg',
 'd68b94105847281e0e66b0fee6e4f3.jpg',
 '8672042617ec09c7a9a18443a42eab.jpg',
 'a849442d81a389bfd136e2a20ebc5a.jpg',
 'a6c1640b35ad762d98387b165d4306.jpg',
 '6286229034c1b6f01879c25c9bd3ec.jpg',
 '46c7c5c6f7439c3e5aff8750fc3955.jpg',
 'e44ea4fc7d8784b7eaae595dd46f47.jpg',
 '1e004663b8c5bfa9590080c07d4025.jpg',
 '17943db0130033ae9153a743565917.jpg',
 'bf0fba4c146e4a984eb81c5d2560f8.jpg',
 'f6a6acf90ab84556ce2493f0431c0e.jpg',
 '327834dc899206d6a8a6e0dd000e80.jpg',
 'b6231b8d31c2dbcd45a2d8bc216f96.jpg',
 'd658e4ff6cdcce90449508ac3bd1ae.jpg',
 '98366e1754efc559249409598d9395.jpg',
 '2a30fa43f48897e8dfb17eabbecf0d.jpg',
 '6c18d288ac91e1d92e7c7fcac3a6b7.jpg',
 '96db177cf1f267e8ed48580caf611f.jpg',
 '0baf4bf0e147c5790c2c3ed33f74bb.jpg',
 '250930190a56e20dab120ce94db2f0.jpg',
 '0f3a9b662ce6b59b067bc3b2b97182.jpg',
 '96f28415fae52e14ce282b5333a0d8.jpg',
 '9674c607f09379482607508e4bbbe8.jpg',
 '3fcc4e8126744bf9a28df8c8425c85.jpg',
 'c1a36e01fc0b4460948c54e967ec39.jpg',
 'b2eaad38a66f6402636e4f1dc7beee.jpg',
 '543cb2d9df927493609a9bf3f45052.jpg',
 '4b6eeccbf0838830b7b4aa731c7269.jpg',
 '7b5233e8eaf238d7882257092b99d0.jpg']

In [ ]:
len(no_vis_classes_val)

In [ ]:
len(no_vis_classes_train)

In [ ]:
# 74f434bf605e8f6f5db0d409142bb7.jpg has mislabeled 3 with 23 as duplicate of 23 - modified manually 